GAN APPROACH

Making flexible predictions : Normalizing the margins to uniform distributions using the empirical Cumulative Distribution Function (CDF), and then training a GAN to make flexible predictions in this normalized space

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import xarray as xr
import xskillscore as xs
import skimage
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils"
os.chdir(path)
import metrics
import dataset

sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

Opening the files :

In [ ]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/HadGEM_driven_COSMO/Present/2003"
os.chdir(path)
filenames = dataset.getfiles()

Normalizing the margins : 

In [ ]:
# Defining generator network
generator = keras.Sequential([layers.Dense(256, input_shape=(latent_dim,)),  # Input layer with 256 units
    layers.LeakyReLU(alpha=0.2),                   # Activation function
    layers.Dense(512),                             # Hidden layer with 512 units
    layers.LeakyReLU(alpha=0.2),                   # Activation function
    layers.Dense(1024),                            # Hidden layer with 1024 units
    layers.LeakyReLU(alpha=0.2),                   # Activation function
    layers.Dense(num_features, activation='tanh')  # Output layer with tanh activation
    ]) 

# Defining discriminator network
discriminator = keras.Sequential([
    layers.Dense(512, input_shape=(num_features,)),  # Input layer with 512 units
    layers.LeakyReLU(alpha=0.2),                     # Activation function
    layers.Dense(256),                               # Hidden layer with 256 units
    layers.LeakyReLU(alpha=0.2),                     # Activation function
    layers.Dense(1, activation='sigmoid')            # Output layer with sigmoid activation
]) 

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Defining discriminator loss function
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

#Defining generator loss function
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
